<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# HubSpot - Get deal brief
<a href="https://app.naas.ai/user-redirect/naas/downloader?url=https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/HubSpot/HubSpot_Get_deal_brief.ipynb" target="_parent"><img src="https://naasai-public.s3.eu-west-3.amazonaws.com/Open_in_Naas_Lab.svg"/></a><br><br><a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=HubSpot+-+Get+deal+brief:+Error+short+description">Bug report</a>

**Tags:** #hubspot #deal #activity #notes #emails #communications #meetings #snippet

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-09-22 (Created: 2023-09-22)

**Description:** This notebook illustrates the process of obtaining a deal brief in HubSpot. It fetches detailed detail information, along with all related activities between you and your sales team. These activities may include emails, notes, meetings, and communications via LinkedIn, WhatsApp, and SMS. The output is conveniently delivered as a text file.

**References:**
- [HubSpot API - Deals](https://developers.hubspot.com/docs/api/crm/deals)
- [HubSpot API - Associations v4](https://developers.hubspot.com/docs/api/crm/associations)
- [HubSpot API - Communications](https://developers.hubspot.com/docs/api/crm/communications)

## Input

### Import libraries

In [1]:
import requests
import naas
from naas_drivers import hubspot
import pandas as pd
import os
pd.set_option('display.max_colwidth', None)

### Setup variables
**Mandatory**

[Get your HubSpot Access token](https://knowledge.hubspot.com/articles/kcs_article/integrations/how-do-i-get-my-hubspot-api-key)
- `hs_access_token`: This variable stores an access token used for accessing the HubSpot API.
- `deal_url`: This variable stores the HubSpot deal URL.

**Optional**
- `deal_properties`: It represents the list of properties to retrieve from your deal
- `associations`: It represents the list of associations to get from your deal
- `output_dir`: This parameter specifies the directory path where the output text file will be stored.

In [2]:
# Mandatory
hs_access_token = naas.secret.get("HS_ACCESS_TOKEN") or "YOUR_HS_ACCESS_TOKEN"
deal_url = "https://app.hubspot.com/contacts/2474088/record/0-3/15220482319/" # "https://app.hubspot.com/contacts/xxxxx/record/0-1/xxxxx"

# Optional
deal_properties = ["hs_object_id", "dealname", "dealstage", 'amount']
associations = ["notes", "emails", "meetings", "communications"]
output_dir = "hubspot"

## Model

### Get deal ID from URL

In [3]:
def get_contact_ID_from_URL(url):
    # Init
    uid = url
    
    # Check if URL is valid
    if not url.startswith("https://app.hubspot.com/contacts/"):
        raise BaseException("HubSpot URL Invalid! It must start by https://app.hubspot.com/contacts/")
    
    # Split URL to get ID
    if "/record/0-3/" in url:
        uid = url.split("/record/0-3/")[-1].split("/")[0]
    return uid

deal_id = get_contact_ID_from_URL(deal_url)
print("Deal ID:", deal_id)

Deal ID: 15220482319


### Get contact details

In [4]:
def retrieve_object_details(
    token,
    object_id,
    object_type,
    properties=None,
):
    # Init
    data = []
    params = {
        "archived": "false"
    }
    
    # Requests
    if properties:
        params["properties"] = properties
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {token}"
    }
    url = f"https://api.hubapi.com/crm/v3/objects/{object_type}/{object_id}"
    
    # Response
    res = requests.get(url, headers=headers, params=params)
    if res.status_code == 200:
        data = res.json().get("properties")
    else:
        print(res.text)
    return pd.DataFrame([data])

def create_activity_df(
    token,
    object_id,
    activity,
    properties_dict=None,
):
    # Init
    properties = [x for x in properties_dict]
    
    # List activities
    df = retrieve_object_details(
        token,
        object_id,
        activity,
        properties
    )
    if len(df) > 0:
        df = df[properties]
        
    if len(df) > 0:
        df = df.rename(columns=properties_dict)
        if 'activity_type' not in df:
            df.insert(loc=1, column="activity_type", value=activity.upper())
    
    return df.reset_index(drop=True)

def get_details(
    hs_access_token,
    hubspot_id,
    properties,
    associations
):
    # Init
    message = "DEAL:\n"
    df = pd.DataFrame()
    
    # Get object
    obj = hubspot.connect(hs_access_token).deals.get(
        hubspot_id,
        hs_properties=properties,
        hs_associations=associations
    )
    
    # Get properties
    obj_properties = obj.get("properties")
    for p in properties:
        message = f"{message}- {p}: {obj_properties.get(p)}\n"
    
    # Get contact associations
    obj_associations = obj.get("associations")
    for a in obj_associations:
        results = obj_associations.get(a).get("results")
        for r in results:
            if a == "communications":
                properties_dict = {
                    "hs_object_id": "activity_hs_id",
                    "hs_timestamp": "activity_date",
                    "hs_communication_channel_type": "activity_type",
                    "hs_body_preview": "activity_content"
                }
            elif a == "meetings":
                properties_dict = {
                    "hs_object_id": "activity_hs_id",
                    "hs_timestamp": "activity_date",
                    "hs_meeting_title": "activity_content"
                }
            else:
                properties_dict = {
                    "hs_object_id": "activity_hs_id",
                    "hs_timestamp": "activity_date",
                    "hs_body_preview": "activity_content"
                }
            association_id = r.get("id")
            
            # Create activity df
            tmp_df = create_activity_df(
                hs_access_token,
                association_id,
                a,
                properties_dict
            )
            df = pd.concat([df, tmp_df])
    
    # Cleaning df
    if len(df) > 0:
        # Exclude empty or None value
        df = df[~(df["activity_content"].astype(str).isin(["None"]))]

        # Format date
        df["activity_date"] = pd.to_datetime(df["activity_date"]).dt.strftime("%Y-%m-%d %H:%M:%S")
        df = df.sort_values(by="activity_date", ascending=False).reset_index(drop=True)
            
    # Create activity message
    message = f"{message}\nACTIVITIES:\n"
    for row in df.itertuples():
        activity_date = row.activity_date
        activity_type = row.activity_type
        activity_content = row.activity_content.replace("\xa0\u200c", "")
        message = f"{message}-{activity_date}: {activity_type} - {activity_content}\n"
    return message, df.reset_index(drop=True)

brief, df_activity = get_details(
    hs_access_token,
    deal_id,
    deal_properties,
    associations
)
print(brief)

DEAL:
- hs_object_id: 15220482319
- dealname: Agicap X Icelex connexion Quadra
- dealstage: 8432674
- amount: None

ACTIVITIES:
-2023-09-19 07:30:00: EMAILS - Bonjour David, Merci de votre retour.  J'ai essayé de me connecter à CEGIDLife mais je n'y ai pas eu accès. (impression d' écran ci-dessous) Pourriez-vous svp me fournir la procédure d'accès? Cordialement, Florent
-2023-09-18 16:46:00: EMAILS - Bonsoir,   Avec mes excuses, la console Admin a été inopérante un temps et je n’ai pu supprimer un compte pour créer le vôtre.   Ci-joint le fichier contenant l’identifiant et le 1er mot de passe à changer à la 1e connexion.   Je créerai votre accès dossier à Quadra demain.   Bien cordialement, David CAMUS Responsable CSP Comptabilité Groupe 11 rue Marcel Sembat - 94140 ALFORTVILLE - FRANCE Tel.: + 33 (0)1 41 79 76 89 – Mob.: +33 (0)6 80 42 57 43 dcamus@icelec.com     Visitez notre site web : www.icegroupe.com   Devez-vous vraiment imprimer ce courriel ? Pensons environnement.   Ce message

## Output

### Save data to txt

In [6]:
def save_txt(output_dir, husbspot_id, txt):
    # Create dirs
    os.makedirs(output_dir, exist_ok=True)
    
    # Create path
    file_path = os.path.join(output_dir, f"{husbspot_id}_deal_brief.txt")
    
    # Save file
    file = open(file_path, 'w')  # Open the file in write mode ('w')
    file.write(txt) # Write the text to the file
    file.close() # Always remember to close the file
    print(f"File successfully saved: {file_path}")
    
save_txt(output_dir, deal_id, brief)

File successfully saved: hubspot/15220482319_deal_brief.txt
